In [2]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="google/gemini-2.0-flash-001",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE"),
    temperature=0.2,
)


In [34]:
from langchain.tools import tool

@tool
def estimate_calories(exercise: str, duration_min: int, weight_kg: float) -> str:
    """This function is used to calculate calories for an exercise. Input the Exercise as string and the duration and the person weight to calculate.
    Currently only supports -> Push Ups, Bench Press and Dumbbell Flyes"""
    met_values = {
        "push ups": 3.8,
        "bench press": 6.0,
        "dumbbell flyes": 4.0,
    }
    met = met_values.get(exercise.lower(), 4.0)
    calories = (met * 3.5 * weight_kg / 200) * duration_min
    return f"Estimated {calories:.1f} calories burned doing {exercise} for {duration_min} mins at {weight_kg}kg."


In [35]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [36]:
from langchain import hub

tools = [estimate_calories, search]

prompt = hub.pull("hwchase17/openai-functions-agent")

In [37]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm,tools,prompt)

In [38]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

chat_histories = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_key="chat_history",
)

In [50]:
agent_with_chat_history.invoke({"input":"So I wanted to start going to the gym, what is the best exercise for me to do on chest day?"},
                               config={"configurable": {"session_id": "user_a"}})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'best chest exercises for beginners'}`


[{'title': 'Chest Workout for Beginners: 10 Easy Exercises to Try (2025)', 'url': 'https://www.mensjournal.com/health-fitness/10-best-chest-exercises-beginners', 'content': 'Best Chest Exercises for Beginners · 1. Pushup · 2. Barbell Bench Press · 3. Medicine Ball Chest Throw · 4. Dumbbell Bench Press.', 'score': 0.9055316}, {'title': 'Beginner Chest Workouts | The Best Exercises For Your Upper And ...', 'url': 'https://us.myprotein.com/thezone/training/beginners-chest-workout-exercises-building-bigger-pecs/', 'content': "The chest muscles are activated through two directional movements â\x80\x94 pushing and pulling. Both movements are important for a well-developed chest, but beginners should focus mainly on pushing movements. Thatâ\x80\x99s because pushing allows for a far greater load to be used, and lifting heavier will enable you to develop a gre

{'input': [HumanMessage(content='So I wanted to start going to the gym, what is the best exercise for me to do on chest day?', additional_kwargs={}, response_metadata={})],
 'output': "Okay, based on the search results, here are some good chest exercises for beginners:\n\n1.  **Push-ups:** A classic bodyweight exercise that's great for building chest strength.\n2.  **Barbell Bench Press:** A popular and effective exercise for overall chest development.\n3.  **Dumbbell Bench Press:** Another great option that allows for a greater range of motion.\n\nIt's recommended to focus on pushing movements as a beginner, as they allow you to lift heavier and develop more muscle mass.\n"}

In [40]:
agent_with_chat_history.invoke({"input":"Cool, so which one of these can you calculate the calories for?"})



> Entering new AgentExecutor chain...
I can calculate the calories for Push Ups, Bench Press, and Dumbbell Flyes.


> Finished chain.


{'input': 'Cool, so which one of these can you calculate the calories for?',
 'output': 'I can calculate the calories for Push Ups, Bench Press, and Dumbbell Flyes.\n'}

In [42]:
agent_with_chat_history.invoke({"input":"Cool, i am 75kg and i would bench press for 6 minutes, calculate the calories for that, also for the same metrics calculate the calories if i were to do push ups and compare the calories for me"})



> Entering new AgentExecutor chain...

Invoking: `estimate_calories` with `{'duration_min': 6, 'exercise': 'Bench Press', 'weight_kg': 75}`


Estimated 47.2 calories burned doing Bench Press for 6 mins at 75.0kg.
Invoking: `estimate_calories` with `{'duration_min': 6, 'weight_kg': 75, 'exercise': 'Push Ups'}`


Estimated 29.9 calories burned doing Push Ups for 6 mins at 75.0kg.Okay, so you would burn approximately 47.2 calories doing Bench Press for 6 minutes at 75kg. Alternatively, you would burn approximately 29.9 calories doing Push Ups for 6 minutes at 75kg. Therefore, you would burn more calories doing bench press.


> Finished chain.


{'input': 'Cool, i am 75kg and i would bench press for 6 minutes, calculate the calories for that, also for the same metrics calculate the calories if i were to do push ups and compare the calories for me',
 'output': 'Okay, so you would burn approximately 47.2 calories doing Bench Press for 6 minutes at 75kg. Alternatively, you would burn approximately 29.9 calories doing Push Ups for 6 minutes at 75kg. Therefore, you would burn more calories doing bench press.\n'}

In [51]:
agent_with_chat_history.invoke({"input":"Hi, I am Youssef"},
                               config={"configurable": {"session_id": "user_a"}})

agent_with_chat_history.invoke({"input":"Hi, I am Ahmed"},
                               config={"configurable": {"session_id": "user_b"}})

agent_with_chat_history.invoke({"input":"What is my Name?"},
                               config={"configurable": {"session_id": "user_a"}})

agent_with_chat_history.invoke({"input":"What is my Name?"},
                               config={"configurable": {"session_id": "user_b"}})



> Entering new AgentExecutor chain...
Hi Youssef, nice to meet you. How can I help you today?


> Finished chain.


> Entering new AgentExecutor chain...
Hello Ahmed, how can I help you today?


> Finished chain.


> Entering new AgentExecutor chain...
Your name is Youssef.


> Finished chain.


> Entering new AgentExecutor chain...
Your name is Ahmed.


> Finished chain.


{'input': [HumanMessage(content='Hi, I am Ahmed', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Ahmed, how can I help you today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my Name?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ahmed.\n'}